In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.image import extract_patches_2d

digits = load_digits()
X, y = digits.images, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

train_mask = np.isin(y_train, [0, 1, 7])
X_train, y_train = X_train[train_mask], y_train[train_mask]

test_mask = np.isin(y_test, [0, 1, 7])
X_test, y_test = X_test[test_mask], y_test[test_mask]

# y_train과 y_test에서 7을 2로 변환
y_train = np.where(y_train == 7, 2, y_train)
y_test = np.where(y_test == 7, 2, y_test)

#
# X_train = X_train.reshape(X_train.shape[0], 16, 4)
# X_test = X_test.reshape(X_test.shape[0], 16, 4)

In [3]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

2024-11-18 12:03:44.830049: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-18 12:03:44.847334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 12:03:44.867773: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 12:03:44.874011: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 12:03:44.892461: I tensorflow/core/platform/cpu_feature_guar

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, average_precision_score
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.001
num_epochs = 10

# X_train과 y_train이 올바르게 변환되고 동일한 첫 번째 차원을 가지는지 확인
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # 채널 차원 추가
y_train = torch.tensor(y_train, dtype=torch.long)  # 다중 클래스 분류를 위해 정수형 변환

X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.long)


# 데이터셋과 데이터 로더 정의
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 3)  # 클래스 개수에 맞게 설정
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 2 * 2)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # CrossEntropyLoss는 로짓을 기대하므로 sigmoid 사용 안 함
        return x

In [5]:
# 모델 초기화
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
def multi_accuracy(preds, y):
    """
    배치별 정확도를 반환합니다 (예: 8/10 개 정답인 경우 0.8 반환)
    """
    pred_soft = torch.log_softmax(preds, dim=1)
    _, pred_index = torch.max(pred_soft, dim=1)
    correct_pred = (pred_index == y).float()  # y와 pred_index 모두 [batch_size] 형태로 맞춤
    acc = correct_pred.sum() / len(correct_pred)
    return acc

In [7]:
# 학습 루프
for epoch in tqdm(range(num_epochs)):
    optimizer.zero_grad()
    
    # 입력 텐서 변환 (추가 차원 제거하여 [batch_size, 1, 28, 28] 형태로 만듦)
    X_tensor = X_train.clone().detach().float()  # [batch_size, 1, 28, 28]
    predictions = model(X_tensor)  # [batch_size, num_classes] 형태의 로짓 출력

    # 레이블이 원-핫 인코딩된 경우 클래스 인덱스로 변환
    label = y_train.clone().detach().long()
    if label.dim() > 1 and label.size(1) > 1:  # 원-핫 인코딩 여부 확인
        label = torch.argmax(label, dim=1)  # 클래스 인덱스로 변환하여 [batch_size] 형태로 맞춤

    # `predictions`와 `label`의 형식 확인
    print(f"Predictions shape: {predictions.shape}, dtype: {predictions.dtype}")
    print(f"Label shape: {label.shape}, dtype: {label.dtype}")

    # 손실 및 정확도 계산
    loss = criterion(predictions, label)
    acc = multi_accuracy(predictions, label)
    print(f'에포크 {epoch + 1}/{num_epochs}, 정확도: {acc:.4f}, 손실: {loss.item():.4f}')
    
    # 역전파 및 옵티마이저 스텝
    loss.backward()
    optimizer.step()


  0%|          | 0/10 [00:00<?, ?it/s]

Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 1/10, 정확도: 0.3469, 손실: 1.1873


/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 10/10 [00:00<00:00, 11.25it/s]

Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 2/10, 정확도: 0.5215, 손실: 1.0992
Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 3/10, 정확도: 0.6435, 손실: 0.8626
Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 4/10, 정확도: 0.9689, 손실: 0.6928
Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 5/10, 정확도: 0.7129, 손실: 0.6346
Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 6/10, 정확도: 0.8684, 손실: 0.5196
Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 7/10, 정확도: 0.9809, 손실: 0.4105
Predictions shape: torch.Size([418, 3]), dtype: torch.float32
Label shape: torch.Size([418]), dtype: torch.int64
에포크 8

In [8]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, average_precision_score

# 테스트 데이터 텐서로 변환
X_tensor = X_test.clone().detach().float()  # X_test 사용, [batch_size, 1, 28, 28] 형태 예상
predictions = model(X_tensor)  # [batch_size, num_classes] 형태의 출력

# 소프트맥스를 적용하여 확률로 변환
predictions = torch.softmax(predictions, dim=1)

# 레이블이 원-핫 인코딩된 경우 클래스 인덱스로 변환
label = y_test.clone().detach().long()  # y_test 사용
if label.dim() > 1 and label.size(1) > 1:  # 원-핫 인코딩 여부 확인
    label = torch.argmax(label, dim=1)  # 클래스 인덱스로 변환하여 [batch_size] 형태로 맞춤

# 손실 계산
loss = criterion(predictions, label)

# sklearn 지표 계산을 위해 predictions와 labels를 numpy 배열로 변환
preds_np = predictions.detach().numpy()  # 그래프에서 분리하여 numpy로 변환
labels_np = label.numpy()  # 레이블도 numpy로 변환

# 예측된 클래스 가져오기
predicted_classes = np.argmax(preds_np, axis=1)

# 정확도 계산
acc = multi_accuracy(predictions, label)

# AUROC (클래스별 one-vs-rest 방식) 계산
auroc = roc_auc_score(labels_np, preds_np, multi_class="ovr")

# labels_np가 원-핫 인코딩이 아닌 클래스 레이블 형식인지 확인
if labels_np.ndim > 1:
    labels_np = np.argmax(labels_np, axis=1)

# 정밀도, 재현율, F1 점수 계산 (다중 클래스의 경우 macro 평균)
precision, recall, f1, _ = precision_recall_fscore_support(labels_np, predicted_classes, average='macro')

# 각 클래스에 대한 AUPRC (정밀-재현 곡선 아래 영역) 계산
auprc = average_precision_score(labels_np, preds_np, average="macro")

# 결과 출력
print(f'\n정확도: {acc}\n')
print(f'AUROC: {auroc}\n')
print(f'정밀도 (macro): {precision}\n')
print(f'재현율 (macro): {recall}\n')
print(f'F1 점수 (macro): {f1}\n')
print(f'AUPRC (macro): {auprc}\n')



정확도: 1.0

AUROC: 1.0

정밀도 (macro): 1.0

재현율 (macro): 1.0

F1 점수 (macro): 1.0

AUPRC (macro): 1.0

